In [5]:
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import os
import time
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
import torch
from torch import nn, optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
import warnings
import shutil
import tqdm # Import tqdm function directly
warnings.filterwarnings("ignore")

In [6]:
def extract_hog_features_recursive(input_dir, force_size = (128, 128), pixels_per_cell=(16, 16), cells_per_block=(2, 2)):
    features = []
    filenames = []
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')
    for root, dirs, files in tqdm.tqdm(os.walk(input_dir)):
        for filename in tqdm.tqdm(files):
            if filename.lower().endswith(supported_formats):
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                # force resized in case feature extraction failed
                img_resized = cv2.resize(img, force_size, interpolation=cv2.INTER_AREA)
                # pixel normalisation
                img_normalised = img_resized.astype(np.float32) / 255.0
                # Extract HOG features
                try:
                    hog_feature = hog(img_normalised, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, feature_vector=True)
                    features.append(hog_feature)
                    rel_path = os.path.relpath(img_path, input_dir)
                    filenames.append(rel_path)
                except Exception as e:
                    print("WARNING: {img_path} Failed with HOG feature extraction!")
                    continue
    hogged = np.array(features)
    return hogged, filenames

# train set:
input_dir = '../CS610_AML_Group_Project/augmented_train_images'
print('====== HOG Extraction Starts! ======')
hogged_train, filenames_train = extract_hog_features_recursive(input_dir)
print('====== HOG Extraction Completed! ======')
print(hogged_train.shape)  # (num_images, hog_feature_dim)

# test set:
input_dir = '../CS610_AML_Group_Project/split_images/test'
print('====== HOG Extraction Starts! ======')
hogged_test, filenames_test = extract_hog_features_recursive(input_dir)
print('====== HOG Extraction Completed! ======')
print(hogged_test.shape)  # (num_images, hog_feature_dim)

====== HOG Extraction Starts! ======


0it [00:00, ?it/s]
100%|██████████| 464/464 [00:09<00:00, 48.54it/s]
51it [07:46,  9.16s/it]


====== HOG Extraction Completed! ======
(20736, 1764)
====== HOG Extraction Starts! ======


0it [00:00, ?it/s]
100%|██████████| 29/29 [00:00<00:00, 47.64it/s]
51it [00:25,  2.03it/s]

====== HOG Extraction Completed! ======
(1296, 1764)


In [7]:
#Labeling
y_train = [f.split(os.sep)[0] for f in filenames_train]

#split data into train_test split
x_train = hogged_train.astype(np.float32)
y_train = np.array(y_train)
y_train, uniques = pd.factorize(y_train)
x_train = pd.DataFrame(x_train, dtype=np.float32)
y_train = pd.Series(y_train, dtype=np.int32)

#Labeling
y_test = [f.split(os.sep)[0] for f in filenames_test]

#split data into train_test split
x_test = hogged_test.astype(np.float32)
y_test = np.array(y_test)
y_test, uniques = pd.factorize(y_test)
x_test = pd.DataFrame(x_test, dtype=np.float32)
y_test = pd.Series(y_test, dtype=np.int32)

In [8]:
print("\n====== Feature Standardisation Started! ======")
scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

print("\n====== Feature Standardisation Completed! ======")
print(f"The Shape for Training Set after Feature Standardisation: {x_train_scaled.shape}")
print(f"The Shape for Testing Set after Feature Standardisation: {x_test_scaled.shape}")


====== Feature Standardisation Started! ======

====== Feature Standardisation Completed! ======
The Shape for Training Set after Feature Standardisation: (20736, 1764)
The Shape for Testing Set after Feature Standardisation: (1296, 1764)


In [9]:
print("\n====== Dimensionality Reduction by PCA Started! ======")
pca = PCA(n_components=0.85, random_state=42)
pca.fit(x_train_scaled)


x_train_pca = pca.transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

print("\n====== Dimensionality Reduction by PCA Completed! ======")
print(f"The Shape for Training Set after Dimensionality Reduction by PCA: {x_train_pca.shape}")
print(f"The Shape for Testing Set after Dimensionality Reduction by PCA: {x_test_pca.shape}")
print(f"The Number of Chosen PCA: {pca.n_components_}")
print(f"The Explained Variance Ratio: {np.sum(pca.explained_variance_ratio_):.4f}")


====== Dimensionality Reduction by PCA Started! ======

====== Dimensionality Reduction by PCA Completed! ======
The Shape for Training Set after Dimensionality Reduction by PCA: (20736, 241)
The Shape for Testing Set after Dimensionality Reduction by PCA: (1296, 241)
The Number of Chosen PCA: 241
The Explained Variance Ratio: 0.8505


In [10]:
with open('model_bank/best_hog_rf_model.pkl', 'rb') as file:
    best_hog_rf = pickle.load(file)
with open('model_bank/best_hog_xgb_model.pkl', 'rb') as file:
    best_hog_xgb = pickle.load(file)
with open('model_bank/best_hog_knn_model.pkl', 'rb') as file:
    best_hog_knn = pickle.load(file)
with open('model_bank/best_cnn_rf_model.pkl', 'rb') as file:
    best_cnn_rf = pickle.load(file)
with open('model_bank/best_cnn_xgb_model.pkl', 'rb') as file:
    best_cnn_xgb = pickle.load(file)
with open('model_bank/best_cnn_knn_model.pkl', 'rb') as file:
    best_cnn_knn = pickle.load(file)

In [11]:
# Predict
y_train_pred = best_hog_rf.predict(x_train_pca)
y_test_pred = best_hog_rf.predict(x_test_pca)

# Function to save metrics
metrics = {"Metric": [], "Average Type": [], "Train": [], "Test": []}
def add_metric(name, avg_type, train_value, test_value):
    metrics["Metric"].append(name)
    metrics["Average Type"].append(avg_type)
    metrics["Train"].append(train_value)
    metrics["Test"].append(test_value)

# Accuracy
add_metric("Accuracy", "N/A",
           accuracy_score(y_train, y_train_pred),
           accuracy_score(y_test, y_test_pred))

# Precision
for avg in ['macro']:
    add_metric("Precision", avg,
               precision_score(y_train, y_train_pred, average=avg),
               precision_score(y_test, y_test_pred, average=avg))

# Recall
for avg in ['macro']:
    add_metric("Recall", avg,
               recall_score(y_train, y_train_pred, average=avg),
               recall_score(y_test, y_test_pred, average=avg))

# F0.5-Score
beta = 0.5 # mis-labelled sneakers are more costly than missing labels
for avg in ['macro']:
    add_metric(f"F{beta}-Score", avg,
               fbeta_score(y_train, y_train_pred, beta=beta, average=avg),
               fbeta_score(y_test, y_test_pred, beta=beta, average=avg))

# Display metrics
hog_rf_metrics = pd.DataFrame(metrics)
pd.set_option('display.precision', 6)
display(hog_rf_metrics)

,Metric,Average Type,Train,Test
0,Accuracy,N/A,0.999035,0.307099
1,Precision,macro,0.998950,0.328319
2,Recall,macro,0.998908,0.301281
3,F0.5-Score,macro,0.998939,0.313624


In [12]:
# Predict
y_train_pred = best_hog_knn.predict(x_train_pca)
y_test_pred = best_hog_knn.predict(x_test_pca)

# Function to save metrics
metrics = {"Metric": [], "Average Type": [], "Train": [], "Test": []}
def add_metric(name, avg_type, train_value, test_value):
    metrics["Metric"].append(name)
    metrics["Average Type"].append(avg_type)
    metrics["Train"].append(train_value)
    metrics["Test"].append(test_value)

# Accuracy
add_metric("Accuracy", "N/A",
           accuracy_score(y_train, y_train_pred),
           accuracy_score(y_test, y_test_pred))

# Precision
for avg in ['macro']:
    add_metric("Precision", avg,
               precision_score(y_train, y_train_pred, average=avg),
               precision_score(y_test, y_test_pred, average=avg))

# Recall
for avg in ['macro']:
    add_metric("Recall", avg,
               recall_score(y_train, y_train_pred, average=avg),
               recall_score(y_test, y_test_pred, average=avg))

# F0.5-Score
beta = 0.5 # mis-labelled sneakers are more costly than missing labels
for avg in ['macro']:
    add_metric(f"F{beta}-Score", avg,
               fbeta_score(y_train, y_train_pred, beta=beta, average=avg),
               fbeta_score(y_test, y_test_pred, beta=beta, average=avg))

# Display metrics
hog_knn_metrics = pd.DataFrame(metrics)
pd.set_option('display.precision', 6)
display(hog_knn_metrics)

  File "d:\Anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "d:\Anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "d:\Anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


,Metric,Average Type,Train,Test
0,Accuracy,N/A,0.998939,0.368827
1,Precision,macro,0.998861,0.379593
2,Recall,macro,0.998801,0.367517
3,F0.5-Score,macro,0.998841,0.371319


In [13]:
# Predict
y_train_pred = best_hog_xgb.predict(x_train_pca)
y_test_pred = best_hog_xgb.predict(x_test_pca)

# Function to save metrics
metrics = {"Metric": [], "Average Type": [], "Train": [], "Test": []}
def add_metric(name, avg_type, train_value, test_value):
    metrics["Metric"].append(name)
    metrics["Average Type"].append(avg_type)
    metrics["Train"].append(train_value)
    metrics["Test"].append(test_value)

# Accuracy
add_metric("Accuracy", "N/A",
           accuracy_score(y_train, y_train_pred),
           accuracy_score(y_test, y_test_pred))

# Precision
for avg in ['macro']:
    add_metric("Precision", avg,
               precision_score(y_train, y_train_pred, average=avg),
               precision_score(y_test, y_test_pred, average=avg))

# Recall
for avg in ['macro']:
    add_metric("Recall", avg,
               recall_score(y_train, y_train_pred, average=avg),
               recall_score(y_test, y_test_pred, average=avg))

# F0.5-Score
beta = 0.5 # mis-labelled sneakers are more costly than missing labels
for avg in ['macro']:
    add_metric(f"F{beta}-Score", avg,
               fbeta_score(y_train, y_train_pred, beta=beta, average=avg),
               fbeta_score(y_test, y_test_pred, beta=beta, average=avg))

# Display metrics
hog_xgb_metrics = pd.DataFrame(metrics)
pd.set_option('display.precision', 6)
display(hog_xgb_metrics)

,Metric,Average Type,Train,Test
0,Accuracy,N/A,0.999035,0.341821
1,Precision,macro,0.998918,0.353889
2,Recall,macro,0.998934,0.337784
3,F0.5-Score,macro,0.998919,0.345981


In [14]:
#Process image data for feature extraction using CNN
input_dir = '../CS610_AML_Group_Project/resized_images'
img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])]) #mean and std based on ImageNet - normalise image data closer to normal distribution
img_dataset = datasets.ImageFolder(input_dir, transform=img_transform)
data_loader = DataLoader(img_dataset, batch_size=32, num_workers=4)

In [15]:
#define function for CNN feature extraction
def cnn_feature_extract(cnn_feature_extractor, data_loader):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #prepare cnn model to use for feature extraction
    cnn_feature_extractor.eval()
    cnn_feature_extractor.fc = torch.nn.Identity() #replace fully connected layer of pretrained cnn with Identity layer
    for para in cnn_feature_extractor.parameters():
        para.requires_grad = False #freeze weights
    #feature extraction
    features_list, labels_list = [], []
    cnn_feature_extractor.to(device)
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            feature = cnn_feature_extractor(images)
            feature = feature.view(feature.size(0),-1) #flatten into (n_samples, n_features) for non-CNN models
            #convert tensors into numpy for fitting into non-CNN models and add into lists
            features_list.append(feature.cpu().numpy())
            labels_list.append(labels.numpy())

    return cnn_feature_extractor, np.vstack(features_list), np.hstack(labels_list)

In [16]:
#initialise and extract features using CNN feature extractor
weights = models.ResNet50_Weights.IMAGENET1K_V2
resnet50_extractor = models.resnet50(weights=weights)
resnet50_extractor, X, y = cnn_feature_extract(resnet50_extractor, data_loader) #X = features, y =labels
#no need labelling as the numpy array is generated from the data_loader

In [17]:
#CNN training and test split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
x_train = pd.DataFrame(x_train, dtype=np.float32)
y_train = pd.Series(y_train, dtype=np.int32)
x_test = pd.DataFrame(x_test, dtype=np.float32)
y_test = pd.Series(y_test, dtype=np.int32)
#same as original flow
print("Number of Samples:", len(y_train))
print("Number of Labels:", len(np.unique(y_train)))
counts = y_train.value_counts()

Number of Samples: 5184
Number of Labels: 50


In [21]:
# Predict
y_train_pred = best_cnn_rf.predict(x_train)
y_test_pred = best_cnn_rf.predict(x_test)

# Function to save metrics
metrics = {"Metric": [], "Average Type": [], "Train": [], "Test": []}
def add_metric(name, avg_type, train_value, test_value):
    metrics["Metric"].append(name)
    metrics["Average Type"].append(avg_type)
    metrics["Train"].append(train_value)
    metrics["Test"].append(test_value)

# Accuracy
add_metric("Accuracy", "N/A",
           accuracy_score(y_train, y_train_pred),
           accuracy_score(y_test, y_test_pred))

# Precision
for avg in ['macro']:
    add_metric("Precision", avg,
               precision_score(y_train, y_train_pred, average=avg),
               precision_score(y_test, y_test_pred, average=avg))

# Recall
for avg in ['macro']:
    add_metric("Recall", avg,
               recall_score(y_train, y_train_pred, average=avg),
               recall_score(y_test, y_test_pred, average=avg))

# F0.5-Score
beta = 0.5 # mis-labelled sneakers are more costly than missing labels
for avg in ['macro']:
    add_metric(f"F{beta}-Score", avg,
               fbeta_score(y_train, y_train_pred, beta=beta, average=avg),
               fbeta_score(y_test, y_test_pred, beta=beta, average=avg))

# Display metrics
cnn_rf_metrics = pd.DataFrame(metrics)
pd.set_option('display.precision', 6)
display(cnn_rf_metrics)

,Metric,Average Type,Train,Test
0,Accuracy,N/A,0.998071,0.368827
1,Precision,macro,0.997978,0.391356
2,Recall,macro,0.997817,0.357844
3,F0.5-Score,macro,0.997926,0.356892


In [19]:
# Predict
y_train_pred = best_cnn_knn.predict(x_train)
y_test_pred = best_cnn_knn.predict(x_test)

# Function to save metrics
metrics = {"Metric": [], "Average Type": [], "Train": [], "Test": []}
def add_metric(name, avg_type, train_value, test_value):
    metrics["Metric"].append(name)
    metrics["Average Type"].append(avg_type)
    metrics["Train"].append(train_value)
    metrics["Test"].append(test_value)

# Accuracy
add_metric("Accuracy", "N/A",
           accuracy_score(y_train, y_train_pred),
           accuracy_score(y_test, y_test_pred))

# Precision
for avg in ['macro']:
    add_metric("Precision", avg,
               precision_score(y_train, y_train_pred, average=avg),
               precision_score(y_test, y_test_pred, average=avg))

# Recall
for avg in ['macro']:
    add_metric("Recall", avg,
               recall_score(y_train, y_train_pred, average=avg),
               recall_score(y_test, y_test_pred, average=avg))

# F0.5-Score
beta = 0.5 # mis-labelled sneakers are more costly than missing labels
for avg in ['macro']:
    add_metric(f"F{beta}-Score", avg,
               fbeta_score(y_train, y_train_pred, beta=beta, average=avg),
               fbeta_score(y_test, y_test_pred, beta=beta, average=avg))

# Display metrics
cnn_knn_metrics = pd.DataFrame(metrics)
pd.set_option('display.precision', 6)
display(cnn_knn_metrics)

,Metric,Average Type,Train,Test
0,Accuracy,N/A,0.998071,0.367284
1,Precision,macro,0.997988,0.389494
2,Recall,macro,0.997819,0.361929
3,F0.5-Score,macro,0.997929,0.377110


In [20]:
# Predict
y_train_pred = best_cnn_xgb.predict(x_train)
y_test_pred = best_cnn_xgb.predict(x_test)

# Function to save metrics
metrics = {"Metric": [], "Average Type": [], "Train": [], "Test": []}
def add_metric(name, avg_type, train_value, test_value):
    metrics["Metric"].append(name)
    metrics["Average Type"].append(avg_type)
    metrics["Train"].append(train_value)
    metrics["Test"].append(test_value)


# Accuracy
add_metric("Accuracy", "N/A",
           accuracy_score(y_train, y_train_pred),
           accuracy_score(y_test, y_test_pred))

# Precision
for avg in ['macro']:
    add_metric("Precision", avg,
               precision_score(y_train, y_train_pred, average=avg),
               precision_score(y_test, y_test_pred, average=avg))

# Recall
for avg in ['macro']:
    add_metric("Recall", avg,
               recall_score(y_train, y_train_pred, average=avg),
               recall_score(y_test, y_test_pred, average=avg))

# F0.5-Score
beta = 0.5 # mis-labelled sneakers are more costly than missing labels
for avg in ['macro']:
    add_metric(f"F{beta}-Score", avg,
               fbeta_score(y_train, y_train_pred, beta=beta, average=avg),
               fbeta_score(y_test, y_test_pred, beta=beta, average=avg))

# Display metrics
cnn_xgb_metrics = pd.DataFrame(metrics)
pd.set_option('display.precision', 6)
display(cnn_xgb_metrics)

,Metric,Average Type,Train,Test
0,Accuracy,N/A,0.997492,0.484568
1,Precision,macro,0.997335,0.486872
2,Recall,macro,0.997212,0.479769
3,F0.5-Score,macro,0.997285,0.480678


## CNN Model

In [3]:
#check if cuda is available to use
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device, "is used")

cuda is used


In [4]:
#Process image data for feature extraction using CNN
input_dir = '../CS610_AML_Group_Project/resized_images'
full_set = datasets.ImageFolder(input_dir)

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees = 15),
    transforms.ColorJitter(brightness=0.1, contrast = 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],std=[0.229,0.224,0.225]) #ImageNet
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
     transforms.Normalize([0.485,0.456,0.406],std=[0.229,0.224,0.225]) #ImageNet
])

train_size = int(0.7*len(full_set))
val_size = int(0.2*len(full_set))
test_size = len(full_set)-train_size-val_size
split_datasets = random_split(
    full_set,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

train_indices = split_datasets[0].indices
val_indices = split_datasets[1].indices
test_indices = split_datasets[2].indices

class CustomSubsetWithTransform(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform

    def __getitem__(self, idx):
        original_idx = self.indices[idx]
        img, label = self.dataset[original_idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.indices)
train_img_dataset = CustomSubsetWithTransform(full_set, train_indices, train_transform)
val_img_dataset = CustomSubsetWithTransform(full_set, val_indices, val_test_transform)
test_img_dataset = CustomSubsetWithTransform(full_set, test_indices, val_test_transform)

batch_size = 32
train_loader = DataLoader(train_img_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_img_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_img_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
print("Data processing done")

Data processing done


In [6]:
#get classes from directory
num_classes = len(full_set.classes)
class_names = full_set.classes
print("Number of classes (Full Set):", num_classes,"\n", full_set.classes)

Number of classes (Full Set): 50 
 ['adidas_forum_high', 'adidas_forum_low', 'adidas_gazelle', 'adidas_nmd_r1', 'adidas_samba', 'adidas_stan_smith', 'adidas_superstar', 'adidas_ultraboost', 'asics_gel-lyte_iii', 'converse_chuck_70_high', 'converse_chuck_70_low', 'converse_chuck_taylor_all-star_high', 'converse_chuck_taylor_all-star_low', 'converse_one_star', 'new_balance_327', 'new_balance_550', 'new_balance_574', 'new_balance_990', 'new_balance_992', 'nike_air_force_1_high', 'nike_air_force_1_low', 'nike_air_force_1_mid', 'nike_air_jordan_11', 'nike_air_jordan_1_high', 'nike_air_jordan_1_low', 'nike_air_jordan_3', 'nike_air_jordan_4', 'nike_air_max_1', 'nike_air_max_270', 'nike_air_max_90', 'nike_air_max_95', 'nike_air_max_97', 'nike_air_max_plus_(tn)', 'nike_air_vapormax_flyknit', 'nike_air_vapormax_plus', 'nike_blazer_mid_77', 'nike_cortez', 'nike_dunk_high', 'nike_dunk_low', 'puma_suede_classic', 'reebok_classic_leather', 'reebok_club_c_85', 'salomon_xt-6', 'vans_authentic', 'vans_

In [3]:
#load best cnn_model
best_model = torch.load("./model_bank/best_cnn_resnet50.pth")
cnn_model = models.resnet50(pretrained=True)

In [ ]:
#load best cnn_model
best_model = torch.load("./model_bank/best_cnn_resnet50.pth")
cnn_model = models.resnet50(pretrained=True)
cnn_model.fc = nn.Linear(cnn_model.fc.in_features, len(class_names))
cnn_model = cnn_model.to(device)
cnn_model.load_state_dict(best_model)
#create dictionary to store metrics
test_holder = {}
test_holder['y_true'], test_holder['y_hat'] = [], []
#start evaluation of model
cnn_model.eval()
test_corrects = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        output = cnn_model(images)
        _, test_preds = torch.max(output, 1)
        test_corrects += (test_preds == labels).sum().item()
        test_holder['y_true'].extend(list(labels.cpu().detach().numpy()))
        test_holder['y_hat'].extend(list(test_preds.cpu().detach().numpy()))


test_y_true_all = test_holder['y_true']
test_y_pred_all = test_holder['y_hat']
test_acc = test_corrects / len(test_loader.dataset)
test_precision = precision_score(test_y_true_all, test_y_pred_all, average='macro')
test_recall = recall_score(test_y_true_all, test_y_pred_all, average='macro')
test_fbeta = fbeta_score(test_y_true_all, test_y_pred_all, beta=beta, average='macro')
test_class_report = classification_report(test_y_true_all, test_y_pred_all)